In [77]:
from __future__ import print_function
import numpy as np
import cv2
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adam 
from keras.preprocessing.image import ImageDataGenerator

In [78]:
num_classes = 6
img_rows,img_cols = 48,48
batch_size = 64

In [79]:
train_data_dir = r"C:\Users\Synergiz\PycharmProjects\OPEN_Emotion_Rec\train"
validation_data_dir = r"C:\Users\Synergiz\PycharmProjects\OPEN_Emotion_Rec\validation"

In [80]:
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

In [81]:
train_generator = train_datagen.flow_from_directory(
					train_data_dir,
					color_mode='grayscale',
					target_size=(img_rows,img_cols),
					batch_size=batch_size,
					class_mode='categorical',
					shuffle=True)

validation_generator = validation_datagen.flow_from_directory(
							validation_data_dir,
							color_mode='grayscale',
							target_size=(img_rows,img_cols),
							batch_size=batch_size,
							class_mode='categorical',
							shuffle=True)

Found 24393 images belonging to 6 classes.
Found 6048 images belonging to 6 classes.


In [82]:
model = Sequential()
# Block-1

model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# Block-2

model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# Block-3

model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# Block-4

model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# Block-5

model.add(Flatten())
model.add(Dense(1024,kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Block-6

model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Block-7

model.add(Dense(num_classes,kernel_initializer='he_normal'))
model.add(Activation('softmax'))

print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_32 (Conv2D)           (None, 48, 48, 32)        320       
_________________________________________________________________
activation_44 (Activation)   (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_40 (Batc (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 48, 48, 32)        9248      
_________________________________________________________________
activation_45 (Activation)   (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_41 (Batc (None, 48, 48, 32)        128       
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 24, 24, 32)       

In [83]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.models import load_model
from time import sleep
from keras.preprocessing.image import img_to_array
from keras.preprocessing import image
import cv2
import numpy as np

In [84]:
checkpoint = ModelCheckpoint(r'C:\Users\Synergiz\PycharmProjects\OPEN_Emotion_Rec\Emo_little-h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True
                          )

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)

callbacks = [earlystop,checkpoint,reduce_lr]


In [85]:
model.compile(loss='categorical_crossentropy',
              optimizer = Adam(lr=0.001),
              metrics=['accuracy'])

nb_train_samples = 24176
nb_validation_samples = 3006
epochs=50

In [10]:
history=model.fit_generator(
                train_generator,
                steps_per_epoch=nb_train_samples//batch_size,
                epochs=epochs,
                callbacks=callbacks,
                validation_data=validation_generator,
                validation_steps=nb_validation_samples//batch_size)

C:\Users\Synergiz\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
377/377 [==============================] - 502s 1s/step - loss: 2.1653 - accuracy: 0.2488 - val_loss: 1.4605 - val_accuracy: 0.4253

Epoch 00001: val_loss improved from inf to 1.46047, saving model to C:\Users\Synergiz\PycharmProjects\OPEN_Emotion_Rec\Emo_little-h5
INFO:tensorflow:Assets written to: C:\Users\Synergiz\PycharmProjects\OPEN_Emotion_Rec\Emo_little-h5\assets
Epoch 2/50
377/377 [==============================] - 489s 1s/step - loss: 1.4241 - accuracy: 0.4416 - val_loss: 1.1574 - val_accuracy: 0.5499

Epoch 00002: val_loss improved from 1.46047 to 1.15738, saving model to C:\Users\Synergiz\PycharmProjects\OPEN_Emotion_Rec\Emo_little-h5
INFO:tensorflow:Assets written to: C:\Users\Synergiz\PycharmProjects\OPEN_Emotion_Rec\Emo_little-h5\assets
Epoch 3/50
377/377 [==============================] - 484s 1s/step - loss: 1.1757 - accuracy: 0.5362 - val_loss: 1.1387 - val_accuracy: 0.5734

Epoch 00003: val_loss improved from 1.15738 to 1.13871, saving model to C:\Users\Syn

In [86]:

#face_classifier = cv2.CascadeClassifier(
#    r'C:\Users\Synergiz\PycharmProjects\OPEN_Emotion_Rec\haarcascade.xml')
classifier = load_model(
    r'C:\Users\Synergiz\PycharmProjects\OPEN_Emotion_Rec\Emo_little-h5.h5')


In [ ]:
#*#* EMOJI #*#*

In [87]:
import tkinter as tk
from tkinter import *
import cv2
from PIL import Image, ImageTk
import os
import threading

In [88]:
model = Sequential()
# Block-1

model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# Block-2

model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# Block-3

model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# Block-4

model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# Block-5

model.add(Flatten())
model.add(Dense(1024,kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Block-6

model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Block-7

model.add(Dense(num_classes,kernel_initializer='he_normal'))
model.add(Activation('softmax'))

In [98]:
classifier = load_model(
    r'C:\Users\Synergiz\PycharmProjects\OPEN_Emotion_Rec\Emo_little-h5.h5')
bounding_box = cv2.CascadeClassifier(
    r'C:\Users\Synergiz\PycharmProjects\OPEN_Emotion_Rec\haarcascade.xml') 

In [106]:
emoji={ 0: "happy" ,1:"angry" ,2:"sad" ,3: "neutral" ,4:"surprised" ,5: "tease" }
cur_path = os.path.dirname(os.path.abspath(r"C:\Users\Synergiz\PycharmProjects\OPEN_Emotion_Rec\emoji"))
emoji_dist={0:cur_path+r"happy.png",
           1:cur_path+r"angry.png",
           2:cur_path+r"sad.png",
           3:cur_path+r"neutral.png",
           4:cur_path+r"surprised.png",
           5:cur_path+r"tease.png"}       #C:\Users\Synergiz\PycharmProjects\OPEN_Emotion_Rec\emoji\

In [115]:
global last_frame1
last_frame1=np.zeros((480,640,3), dtype=np.uint8)
global cap1
show_text=[0]
global frame_number



def show_subject():
    cap1 = cv2.VideoCapture(0)
    if not cap1.isOpened():
        print("Can't open the camera")
    global frame_number
    length = int(cap1.get(cv2.CAP_PROP_FRAME_))
    frame_number += 1
    if frame_number >= length:
        exit()
    cap1.set(1, frame_number)
    flag1, frame1 =cap1.read()
    frame1 =cv2.resize(frame1,(600,500))
    bounding_box = cv2.CascadeClassifier(
    r'C:\Users\Synergiz\PycharmProjects\OPEN_Emotion_Rec\haarcascade.xml') 
    gray_frame = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    num_faces = bounding_box.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)
    
    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame1, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
        roi_gray_frame = gray_frame[y:y +h, x:x +w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48,48)), -1), 0)
        prediction = classifier.predict(cropped_img)  # prediction = classifier.predict(roi)[0]
        maxindex = int(np.argmax(prediction))
        cv2.putText(frame1, emoji[maxindex],(x+20, y-60),cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        show_text[0] = maxindex
        
        if flag1 is None:
            print("Major error! ")
        elif flag1:
            global last_frame1
            last_frame1 = frame1.copy()
            pic = cv2.cvtColor(last_frame1, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(pic)
            imgtk = ImageTk.PhotoImage(image=img)
            lmain.imgtk = imgtk
            lmain.configure(image=imgtk)
            root.update()
            lmain.after(10, show_subject)
   
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break


In [116]:

def show_smiley():
    frame2.cv2.imread(emoji_dist[show_text[0]])
    pic2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2RGB)
    img2 = Image.fromarray(frame2)
    imgtk2= ImageTk.PhotoImage(image=img2)
    lmain2.imgtk2=imgtk2
    lmain3.configure(text=emoji[show_text[0]], font=('arial', 45, 'bold'))
    lmain2.configure(image=imgtk2)
    root.update()
    lmain2.after(10, show_smiley)

In [113]:
if '__name__' == '__main__':
    frame_number=0
    root=tk.Tk()
    lmain = tk.Label(master=root, padx=50, bd=10)
    lmain2 = tk.Label(master=root, bd=10)
    lmain3 = tk.Label(master=root, bd=10, fg="#CDCDCD", bg='black')
    lmain.pack(side=LEFT)
    lmain.place(x=50, y=250)
    lmain3.pack()
    lmain3.place(x=960, y=250)
    lmain2.pack(side=RIGHT)
    lmain.place(x=900, y=350)
    
    root.title("Photo Emoji")
    root.goemetry("1400x900+100+10")
    root['bg']='black'
    exitButton = Button(root,text='Quit', fg="red", command=root.destroy, font=('arial', 25, 'bold')).pack(side = BOTTOM)
    threading.Thread(target = show_subject).start()
    threading.Thread(target = show_smiley).start()
    root.mainloop
    
